In [6]:
// The ownership of a variable follows the same pattern every time: assigning a value to another variable moves it. 
// When a variable that includes data on the heap goes out of scope, the value will be cleaned up by drop unless ownership of the data has been moved to another variable.

// While this works, taking ownership and then returning ownership with every function is a bit tedious. 
// What if we want to let a function use a value but not take ownership? 
// It’s quite annoying that anything we pass in also needs to be passed back if we want to use it again, in addition to any data resulting from the body of the function that we might want to return as well.

// Rust does let us return multiple values using a tuple

// But this is too much ceremony and a lot of work for a concept that should be common. Luckily for us, Rust has a feature for using a value without transferring ownership, called references.

fn main() {
    let s1 = String::from("hello");

    let (s2, len) = calculate_length(s1);

    println!("The length of '{}' is {}.", s2, len);
}

fn calculate_length(s: String) -> (String, usize) {
    let length = s.len(); // len() returns the length of a String

    (s, length)
}

main()

The length of 'hello' is 5.


()

In [7]:
 // A reference is like a pointer in that it’s an address we can follow to access the data stored at that address; 
// that data is owned by some other variable. Unlike a pointer, a reference is guaranteed to point to a valid value of a particular type for the life of that reference.

fn main() {
    let s1 = String::from("hello");

    let len = calculate_length(&s1);

    println!("The length of '{}' is {}.", s1, len);
}

fn calculate_length(s: &String) -> usize { // s is a reference to a String
    s.len()
} // Here, s goes out of scope. But because it does not have ownership of what
  // it refers to, it is not dropped.

main()

The length of 'hello' is 5.


()

we take &String rather than String. These ampersands represent references, and they allow you to refer to some value without taking ownership of it. Figure 4 depicts this concept.

<img src="imgs/img4.svg" width="70%" height="70%">
Figure 4: A diagram of &String s pointing at String s1
</br>
</br>

The scope in which the variable s is valid is the same as any function parameter’s scope, but the value pointed to by the reference is not dropped when s stops being used, because s doesn’t have ownership. When functions have references as parameters instead of the actual values, we won’t need to return the values in order to give back ownership, because we never had ownership.

We call the action of creating a reference borrowing. As in real life, if a person owns something, you can borrow it from them. When you’re done, you have to give it back. You don’t own it.

In [8]:
fn main() {
    let s = String::from("hello");

    change(&s);
}

// Just as variables are immutable by default, so are references. We’re not allowed to modify something we have a reference to.
fn change(some_string: &String) {
    some_string.push_str(", world");
}

main()

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

In [9]:
fn main() {
    let mut s = String::from("hello");

    change(&mut s);
}

fn change(some_string: &mut String) {
    some_string.push_str(", world");
}

// First we change s to be mut. Then we create a mutable reference with &mut s where we call the change function, and update the function signature to accept a mutable reference with some_string: &mut String. 
// This makes it very clear that the change function will mutate the value it borrows.

main()

()

In [10]:
fn main() {
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;

    println!("{}, {}", r1, r2);
}

// Mutable references have one big restriction: if you have a mutable reference to a value, you can have no other references to that value. 
// This code that attempts to create two mutable references to s will fail:
main()

Error: cannot borrow `s` as mutable more than once at a time

This error says that this code is invalid because we cannot borrow s as mutable more than once at a time. The first mutable borrow is in r1 and must last until it’s used in the println!, but between the creation of that mutable reference and its usage, we tried to create another mutable reference in r2 that borrows the same data as r1.

The restriction preventing multiple mutable references to the same data at the same time allows for mutation but in a very controlled fashion. It’s something that new Rustaceans struggle with because most languages let you mutate whenever you’d like. The benefit of having this restriction is that Rust can prevent data races at compile time. A data race is similar to a race condition and happens when these three behaviors occur:

* Two or more pointers access the same data at the same time.
* At least one of the pointers is being used to write to the data.
* There’s no mechanism being used to synchronize access to the data.

In [11]:
// As always, we can use curly brackets to create a new scope, allowing for multiple mutable references, just not simultaneous ones:
fn main() {
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
}

main()

()

In [12]:
// Rust enforces a similar rule for combining mutable and immutable references. This code results in an error:
fn main() {
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    let r3 = &mut s; // BIG PROBLEM

    println!("{}, {}, and {}", r1, r2, r3);
}

// Whew! We also cannot have a mutable reference while we have an immutable one to the same value.

// Users of an immutable reference don’t expect the value to suddenly change out from under them! 
// However, multiple immutable references are allowed because no one who is just reading the data has the ability to affect anyone else’s reading of the data.
main()

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

In [13]:
fn main() {
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    println!("{} and {}", r1, r2);
    // variables r1 and r2 will not be used after this point

    let r3 = &mut s; // no problem
    println!("{}", r3);
}

// Note that a reference’s scope starts from where it is introduced and continues through the last time that reference is used. 
// For instance, this code will compile because the last usage of the immutable references, the println!, occurs before the mutable reference is introduced:

// The scopes of the immutable references r1 and r2 end after the println! 
// where they are last used, which is before the mutable reference r3 is created. 
// These scopes don’t overlap, so this code is allowed: the compiler can tell that the reference is no longer being used at a point before the end of the scope.
main()

hello and hello
hello


()

In languages with pointers, it’s easy to erroneously create a dangling pointer—a pointer that references a location in memory that may have been given to someone else—by freeing some memory while preserving a pointer to that memory. In Rust, by contrast, the compiler guarantees that references will never be dangling references: if you have a reference to some data, the compiler will ensure that the data will not go out of scope before the reference to the data does.

In [14]:
fn main() {
    let reference_to_nothing = dangle();
}

fn dangle() -> &String { // dangle returns a reference to a String

    let s = String::from("hello"); // s is a new String

    &s // we return a reference to the String, s
} // Here, s goes out of scope, and is dropped. Its memory goes away.

main()

Error: missing lifetime specifier

In [15]:
// The solution here is to return the String directly:
fn main() {
    let string = no_dangle();
}

fn no_dangle() -> String {
    let s = String::from("hello");

    s
}

// This works without any problems. Ownership is moved out, and nothing is deallocated.
main()

()